In [ ]:
!pip install rank_bm25
!pip install unidecode
!pip install --upgrade gensim
!pip install pandas
!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import string
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import gensim
from gensim.summarization.bm25 import BM25

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/p1ntsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Abertura DataSets

In [9]:
with open('../Traducao/Traducao WHO/who_traduzido.csv','r') as f:
    df_who = pd.read_csv(f)
df_who.head()

,perguntas,respostas
0,O que é COVID-19?,COVID-19 é a doença causada por um novo corona...
1,Quais são os sintomas da COVID-19?,Os sintomas mais comuns de COVID-19 são Febre ...
2,O que acontece com as pessoas que obtêm COVID-19?,"Entre os que desenvolvem sintomas, a maioria (..."
3,Quem corre maior risco de contrair doenças gra...,Pessoas com 60 anos ou mais e aqueles com prob...
4,Existem efeitos a longo prazo do COVID-19?,"Algumas pessoas que tiveram COVID-19, quer ten..."


In [5]:
with open('../Traducao/Traducao COVID-Q/traduzido_final_master_dataset.csv','r') as f:
    df_covid_q = pd.read_csv(f)
df_covid_q.head()

,pergunta,resposta
0,o covid-19 terminará em breve?,"1º de maio, eu acho, é completamente irreal, d..."
1,o covid-19 terminará?,"1º de maio, eu acho, é completamente irreal, d..."
2,quando covid-19 vai acabar?,"1º de maio, eu acho, é completamente irreal, d..."
3,quando o bloqueio covid-19 termina?,"1º de maio, eu acho, é completamente irreal, d..."
4,o covid-19 irá embora?,"1º de maio, eu acho, é completamente irreal, d..."


In [10]:
col = df_who.columns
df_covid_q.columns = col
result = pd.concat([df_who,df_covid_q])
print('Tamanho: {}'.format(result.shape[0]))

Tamanho: 1975


# Pre-processamento

In [16]:
perguntas = result['perguntas'].values
for i in range(perguntas.size):
    perguntas[i] = unidecode(perguntas[i]).lower()
result['perguntas'] = perguntas
result.head()

,perguntas,respostas
0,o que e covid-19?,COVID-19 é a doença causada por um novo corona...
1,quais sao os sintomas da covid-19?,Os sintomas mais comuns de COVID-19 são Febre ...
2,o que acontece com as pessoas que obtem covid-19?,"Entre os que desenvolvem sintomas, a maioria (..."
3,quem corre maior risco de contrair doencas gra...,Pessoas com 60 anos ou mais e aqueles com prob...
4,existem efeitos a longo prazo do covid-19?,"Algumas pessoas que tiveram COVID-19, quer ten..."


In [17]:
print(("Dados duplicados: %d")%(result.duplicated(['perguntas']).sum()))
result.drop_duplicates(['perguntas'],inplace=True)
result.reset_index(inplace= True,drop = True)
print("Shape dataset: {}".format(result.shape))

Dados duplicados: 53
Shape dataset: (1922, 2)


In [19]:
perguntas = result['perguntas']

In [20]:
document = []
for pergunta,i in zip(perguntas,range(perguntas.size)):
    try:
        tokens = nltk.word_tokenize(pergunta,language = 'portuguese')
        tokens = [w for w in tokens if not w in string.punctuation]
        document.append(tokens)
    except:
        print(pergunta, i)
document = np.array(document)

<ipython-input-20-c46f98ed8fea>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  document = np.array(document)


# Salvando amostra

In [22]:
sample = np.random.choice(document,size=50,replace=False)

In [27]:
with open('./Amostra Analise Dataset/amostra.json','w') as outfile:
    json.dump(sample.tolist(),outfile)

# Abrindo amostra

In [28]:
with open('./Amostra Analise Dataset/amostra.json','r') as f:
    sample = json.load(f)

# Utilizando BM25

In [29]:
bm25 = BM25(document)

In [38]:
posicao =  7
try:
    query = sample[posicao]
    if("?" not in query):
        print('Query:'," ".join(query) + '?','\n')
    else:
        print(" ".join(query),'\n')
    score = bm25.get_scores(query)
    top_n = np.argsort(score)[::-1][2:5] # Apresentar as 3 melhores assimilações
    for i in top_n:
        print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25
except:
    print("Posicao invalida")

Query: onde posso encontrar as informacoes mais recentes sobre o covid-19? 

Pergunta: onde posso encontrar mais conselhos sobre como avaliar e gerenciar os riscos a saude em reunioes de massa internacionais? | Score BM25: 13.95
Pergunta: onde posso fazer o teste de covid-19? | Score BM25: 10.95
Pergunta: onde posso comprar desinfetante para as maos? | Score BM25: 10.79
